In [1]:
PLOT_DBG = True

PLOT_OUT = True



import pickle;
import numpy as np;
import pandas as pd;
import matplotlib;
import matplotlib.pyplot as plt;
from helper import *;
import os.path;
import hashlib;


import seaborn as sns;
sns.set_theme(style="whitegrid");
def simple_plot():
    #simpple plot:
    a=[1,2,3];
    b=[1,2,3];
    plt.figure(9948148,figsize=(3,00.1));
    plt.plot(a,b);
    plt.show();

if PLOT_OUT:
    %matplotlib tk
    matplotlib.use('TkAgg')

elif PLOT_DBG:
    %matplotlib inline
    simple_plot();
    %matplotlib notebook

else:
    %matplotlib inline
    %matplotlib inline


ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'headless' is currently running

In [3]:
def split_and_extract(group,server_port,split_time):
    df_splt = group[group['tcp.srcport'] == server_port]
    df_dplt = group[group['tcp.dstport'] == server_port]
    df_src_split = split_comm_flows(df_splt, split_time)
    df_dst_split = split_comm_flows(df_dplt, split_time)
    df_src_feat = extract_feat_list(df_src_split)
    df_dst_feat = extract_feat_list(df_dst_split)
    return {'feat':{'ps':df_src_feat,'client':df_dst_feat},'split':{'ps':df_splt,'client':df_dplt},'original':group}

In [4]:

#SPLIT_TIME = 0.1
#SPLIT_TIME = 5
SPLIT_TIME = 50


#ANALYZED_PCAP = "/home/antonio/TRUSTroke/FLBenchComm/Test/flower/flower_linuxrouter/output/Tflower_R10_C5_t20230922182842/20230922182842_ps.pcap"
ANALYZED_PCAP = "/home/antonio/TRUSTroke/FLBenchComm/Test/flower/flower_linuxrouter_bidi/output/Tflower_R10_C5_t20230926220718/20230926220718_ps.pcap"
ANALYZED_PCAP_1 = "/home/antonio/TRUSTroke/FLBenchComm/Test/flower/flower_linuxrouter_rest/output/Tflower_R10_C5_t20230926181659/20230926181659_ps.pcap"
ANALYZED_PCAP_2 = "/home/antonio/TRUSTroke/FLBenchComm/Test/flower/flower_linuxrouter_rere/output/Tflower_R10_C5_t20230926172055/20230926172055_ps.pcap"

file_list = [ANALYZED_PCAP, ANALYZED_PCAP_1,ANALYZED_PCAP_2]
type_list = ['gRPC','REST','gRPC rere']
port_list = [9092,9093,9092]

hash_object = hashlib.sha1(''.join(file_list).join(str(SPLIT_TIME)).encode())
hash_str = hash_object.hexdigest()#[0:10]
print(hash_str)

if os.path.isfile(f'flow_{hash_str}.pkl'):
    with open(f'flow_{hash_str}.pkl', 'rb') as f:
        flow_list = pickle.load(f)
else:
    file_list=check_file_list(file_list)
    flow_list = []
    for pcap, protocol_type,port in zip(file_list, type_list,port_list):
        set_server_port(port)
        df=(load_df(pcap))
        flows = []
        for elem in extract_flows(df, port,'10.0.0.100'):
            flows.append(split_and_extract(elem,port,SPLIT_TIME))
        flow_list.append({'type':protocol_type,'flows':flows,'df':df,'path':pcap})
    
    with open(f'flow_{hash_str}.pkl', 'wb') as f:
        pickle.dump(flow_list, f)
    

00b63d85e563f73906541458b118ad5d6bc51df1


In [5]:
FLOWN = 0

pr_type = flow_list[FLOWN]['type']
df_splt = flow_list[FLOWN]['flows'][0]['split']['ps']
df_dplt = flow_list[FLOWN]['flows'][0]['split']['client']
df_src_feat = flow_list[FLOWN]['flows'][0]['feat']['ps']
df_dst_feat = flow_list[FLOWN]['flows'][0]['feat']['client']

# src = str(str(int(df_splt['tcp.srcport'].iloc[0]))+'->'+str(int(df_splt['tcp.dstport'].iloc[0])))
# dst = str(str(int(df_splt['tcp.srcport'].iloc[0]))+'<-'+str(int(df_splt['tcp.dstport'].iloc[0])))

src = 'ps->client'
dst = 'client->ps'

title = flow_list[FLOWN]['type']+' flow: ps->client'
plt.figure(title)
plt.title(title)
plt.scatter(df_splt['frame.time'], df_splt['tcp.len'],label=src)
plt.scatter(df_dplt['frame.time'], df_dplt['tcp.len'],label=dst)
plt.xlabel('time [s]')
plt.ylabel('Bytes')
plt.legend()

title = 'Comm. Sum '+flow_list[FLOWN]['type']+' flow: ps->client'
plt.figure(title)
plt.title(title)
plt.scatter(df_src_feat['frame.time.mean'], df_src_feat['frame.len.sum'],label=f"{src}")
plt.scatter(df_dst_feat['frame.time.mean'], df_dst_feat['frame.len.sum'],label=f"{dst}")
plt.xlabel('time [s]')
plt.ylabel('Bytes')
plt.legend()

# plt.figure('tcp Sum '+pr_type+' flow:'+str(int(df_splt['tcp.srcport'].iloc[0]))+':'+str(int(df_splt['tcp.dstport'].iloc[0])))
# plt.title('tcp Sum '+pr_type+' flow:'+str(int(df_splt['tcp.srcport'].iloc[0]))+':'+str(int(df_splt['tcp.dstport'].iloc[0])))
# plt.scatter(df_src_feat['frame.time.mean'], df_src_feat['tcp.len.sum'],label=f"{src}")
# plt.scatter(df_dst_feat['frame.time.mean'], df_dst_feat['tcp.len.sum'],label=f"{dst}")
# plt.legend()

plt.show()


In [6]:
# src = "ps->client"
# dst = "client->ps"
# 
# #filter only big flows
# df_src_feat_fi = df_src_feat[df_src_feat['frame.len.sum'] > 1000]
# df_dst_feat_fi = df_dst_feat[df_dst_feat['frame.len.sum'] > 1000]
# 
# print_feat(df_src_feat_fi,'frame.time.deltat',100,2,None,None,label=src,show=False)
# print_feat(df_dst_feat_fi,'frame.time.deltat',100,2,None,None,label=dst)
# 
# # 
# print_feat(df_src_feat_fi,'frame.time.deltat',100,2,log=True,label=src,plt_hist=False)
# print_feat(df_dst_feat_fi,'frame.time.deltat',100,2,log=True,label=dst,plt_hist=False)
# 
# 
# print_feat(df_src_feat_fi,'frame.len.sum',30,1,label=src,show=False)
# print_feat(df_dst_feat_fi,'frame.len.sum',30,1,label=dst)
# 
# 
# print_feat(df_src_feat,'frame.len.mean',30,1,label=src,show=False)
# print_feat(df_dst_feat,'frame.len.mean',30,1,label=dst)
# 
# print_feat(df_splt,'tcp.len',30,1,None,None,label=src,show=False)
# print_feat(df_dplt,'tcp.len',30,1,None,None,label=dst)


In [94]:
# df_src_feat_list = []
# df_src_feat_list_1 = []
# df_src_feat_list_2 = []
# 
# for elem in  flow_list[0]['flows']:
#     tmp = elem['feat']['ps']
#     df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 0])
# 
# for elem in  flow_list[1]['flows']:
#     tmp = elem['feat']['ps']
#     df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 0])
# 
# for elem in  flow_list[2]['flows']:
#     tmp = elem['feat']['ps']
#     df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 0])
# 
# 
# print_feat_list(df_src_feat_list,'frame.time.deltat',500,1,label=flow_list[0]['type'],log=False,plt_hist=False,xlabel='time Delta [ms]',ylabel='Density',fig_name='Time to send model parameters from parameter server to client',show=False)
# print_feat_list(df_src_feat_list_1,'frame.time.deltat',500,1,label=flow_list[1]['type'],log=False,plt_hist=False,fig_name='Time to send model parameters from parameter server to client',show=False)
# print_feat_list(df_src_feat_list_2,'frame.time.deltat',500,1,label=flow_list[1]['type'],log=False,plt_hist=False,fig_name='Time to send model parameters from parameter server to client')
# 
# df_src_feat_list = []
# df_src_feat_list_1 = []
# df_src_feat_list_2 = []
# 
# for elem in  flow_list[0]['flows']:
#     tmp = elem['feat']['client']
#     df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 0])
# 
# for elem in  flow_list[1]['flows']:
#     tmp = elem['feat']['client']
#     df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 0])
# 
# for elem in  flow_list[2]['flows']:
#     tmp = elem['feat']['client']
#     df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 0])
# 
# print_feat_list(df_src_feat_list,'frame.time.deltat',500,1,label=flow_list[0]['type'],log=False,plt_hist=False,xlabel='time Delta [ms]',ylabel='Density',fig_name='Time to send model parameters from client to parameter server',show=False)
# print_feat_list(df_src_feat_list_1,'frame.time.deltat',500,1,label=flow_list[1]['type'],log=False,plt_hist=False,xlabel='time Delta [ms]',ylabel='Density',fig_name='Time to send model parameters from client to parameter server',show=False)
# print_feat_list(df_src_feat_list_2,'frame.time.deltat',500,1,label=flow_list[2]['type'],log=False,plt_hist=False,xlabel='time Delta [ms]',ylabel='Density',fig_name='Time to send model parameters from client to parameter server')


In [95]:
# df_src_feat_list = []
# df_src_feat_list_1 = []
# df_src_feat_list_2 = []
# 
# for elem in  flow_list[0]['flows']:
#     tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
#     df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 0])
# 
# for elem in  flow_list[1]['flows']:
#     tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
#     df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 0])
# 
# for elem in  flow_list[2]['flows']:
#     tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
#     df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 0])
# 
# 
# print_feat_list(df_src_feat_list,'frame.time.deltat',500,2,label=flow_list[0]['type'],log=False,plt_hist=False,fig_name='PDF on Time to send model parameters',xlabel='Time delta [ms]',ylabel='Density',show=False)
# print_feat_list(df_src_feat_list_1,'frame.time.deltat',500,2,label=flow_list[1]['type'],log=False,plt_hist=False,fig_name='PDF on Time to send model parameters',xlabel='Time delta [ms]',ylabel='Density',show=False)
# print_feat_list(df_src_feat_list_2,'frame.time.deltat',500,2,label=flow_list[2]['type'],log=False,plt_hist=False,fig_name='PDF on Time to send model parameters')

In [96]:
df_src_feat_list = []
df_src_feat_list_1 = []
df_src_feat_list_2 = []

for elem in  flow_list[0]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 0])
    
for elem in  flow_list[1]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 0])

for elem in  flow_list[2]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 0])

df_src_feat_list_all = []
df_src_feat_list_all_1 = []
df_src_feat_list_all_2= []

for elem in df_src_feat_list:
    df_src_feat_list_all = np.append(df_src_feat_list_all,elem['frame.time.deltat'])

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1 = np.append(df_src_feat_list_all_1,elem['frame.time.deltat'])

for elem in df_src_feat_list_2:
    df_src_feat_list_all_2 = np.append(df_src_feat_list_all_2,elem['frame.time.deltat'])

plt.figure('PDF on Time to send model parameters')
for elem,label in zip([df_src_feat_list_all, df_src_feat_list_all_1,df_src_feat_list_all_2],[flow_list[0]['type'],flow_list[1]['type'], flow_list[2]['type']]):
    
    sns.distplot(elem, hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 1.5}, label = label)

plt.ylim(0, None)
plt.legend(prop={'size': 12}, title = 'Protocol')
plt.title('PDF on Time to send model parameters'); 
plt.xlabel('Time delta [ms]'); plt.ylabel('Density');


/tmp/ipykernel_86901/3882459298.py:33: UserWarning:



`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751


/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/distributions.py:2511: FutureWarning:



`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.


/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/home/antonio/TRUSTroke/FLBenchComm/Fra

In [97]:
df_src_feat_list = []
df_src_feat_list_1 = []
df_src_feat_list_2 = []

for elem in  flow_list[0]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 0])
    
for elem in  flow_list[1]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 0])

for elem in  flow_list[2]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 0])

df_src_feat_list_all = []
df_src_feat_list_all_1 = []
df_src_feat_list_all_2= []

for elem in df_src_feat_list:
    df_src_feat_list_all = np.append(df_src_feat_list_all,elem['frame.time.deltat'])

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1 = np.append(df_src_feat_list_all_1,elem['frame.time.deltat'])

for elem in df_src_feat_list_2:
    df_src_feat_list_all_2 = np.append(df_src_feat_list_all_2,elem['frame.time.deltat'])
    

fig_name = 'Sending model parameters distribution'

# Creating plot
plt.figure(fig_name)
plt.title(fig_name)
# sns.boxplot([df_src_feat_list_all,df_src_feat_list_all_1])
# plt.show()




sns.violinplot(data=[df_src_feat_list_all,df_src_feat_list_all_1,df_src_feat_list_all_2,], palette='Set3')
plt.xticks([0, 1, 2], [flow_list[0]['type'],flow_list[1]['type'], flow_list[2]['type']])
plt.ylabel('time Delta [ms]')
plt.show()


In [98]:
df_src_feat_list = []
df_src_feat_list_1 = []
df_src_feat_list_2 = []

name = 'tcp.len'
for elem in  flow_list[0]['flows']:
    tmp = pd.concat([elem['split']['ps'], elem['split']['client']])
    df_src_feat_list.append(tmp[tmp['frame.len'] > 100])
    
for elem in  flow_list[1]['flows']:
    tmp = pd.concat([elem['split']['ps'], elem['split']['client']])
    df_src_feat_list_1.append(tmp[tmp['frame.len'] > 100])
    

for elem in  flow_list[2]['flows']:
    tmp = pd.concat([elem['split']['ps'], elem['split']['client']])
    df_src_feat_list_2.append(tmp[tmp['frame.len'] > 100])

df_src_feat_list_all = []
df_src_feat_list_all_1 = []
df_src_feat_list_all_2 = []

for elem in df_src_feat_list:
    df_src_feat_list_all = np.append(df_src_feat_list_all,elem[name])

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1 = np.append(df_src_feat_list_all_1,elem[name])

for elem in df_src_feat_list_2:
    df_src_feat_list_all_2 = np.append(df_src_feat_list_all_2,elem[name])

fig_name = 'violin plot packets len' 
# Creating plot
plt.figure(fig_name)
plt.title(fig_name)
# sns.boxplot([df_src_feat_list_all,df_src_feat_list_all_1])
# plt.show()

sns.violinplot(data=[df_src_feat_list_all,df_src_feat_list_all_1,df_src_feat_list_all_2], palette='Set3')
plt.xticks([0, 1,2], [flow_list[0]['type'], flow_list[1]['type'], flow_list[2]['type']])
plt.ylabel('Frame Len [Bytes]')

plt.show()

In [99]:
RANGE = [4, 11]
df_src_feat_list = []
df_src_feat_list_1 = []
df_src_feat_list_2 = []

for elem in  flow_list[0]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[1]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 5000])
    
for elem in  flow_list[2]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 5000])
    
df_src_feat_list_all = []
df_src_feat_list_all_1 = []
df_src_feat_list_all_2 = []

for elem in df_src_feat_list:
    df_src_feat_list_all.append(df_src_feat_list_all)

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1.append(df_src_feat_list_all_1)
    
for elem in df_src_feat_list_2:
    df_src_feat_list_all_2.append(df_src_feat_list_all_2)
    
name = 'frame.time.deltat'

x_m,std_m = extract_mean_std_feat_list(df_src_feat_list,name)
x_m1,std_m1 = extract_mean_std_feat_list(df_src_feat_list_1,name)
x_m2,std_m2 = extract_mean_std_feat_list(df_src_feat_list_2,name)

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': RANGE}},
    value = x_m,
    delta = {'reference': 7.02},
    domain = {'x': [0.1, 1], 'y': [0, 0.25]},
    title = {'text': flow_list[0]['type']+' [ms]'}))
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': RANGE}},
    value = x_m1,
    delta = {'reference': 7.02},
    domain = {'x': [0.1, 1], 'y': [0.35, 0.6]},
    title = {'text': flow_list[1]['type']+' [ms]'}))
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': RANGE}},
    value = x_m2,
    delta = {'reference': 7.02},
    domain = {'x': [0.1, 1], 'y': [0.7, 0.95]},
    title = {'text': flow_list[2]['type']+' [ms]'}))

# update layout
fig.update_layout(height=400,title_text="Time delay REST vs gRPC in [ms]")

fig.show()




In [100]:
# Define the data
x_labels = [flow_list[0]['type'], flow_list[1]['type'],flow_list[2]['type']]

RANGE = [4, 11]
df_src_feat_list = []
df_src_feat_list_1 = []
df_src_feat_list_2 = []

for elem in  flow_list[0]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[1]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 5000])
    
for elem in  flow_list[2]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 5000])
    
df_src_feat_list_all = []
df_src_feat_list_all_1 = []
df_src_feat_list_all_2 = []

for elem in df_src_feat_list:
    df_src_feat_list_all.append(df_src_feat_list_all)

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1.append(df_src_feat_list_all_1)
    
for elem in df_src_feat_list_2:
    df_src_feat_list_all_2.append(df_src_feat_list_all_2)
    
name = 'frame.time.deltat'

x_m,std_m = extract_mean_std_feat_list(df_src_feat_list,name)
x_m1,std_m1 = extract_mean_std_feat_list(df_src_feat_list_1,name)
x_m2,std_m2 = extract_mean_std_feat_list(df_src_feat_list_2,name)

x_m_values = [x_m,x_m1,x_m2]
colors = ['blue', 'red', 'green']
std_m_values = [std_m,std_m1,std_m2]
# Create the bar graph plot
# fig, ax = plt.subplots()
# for i in range(len(x_labels)):
#     ax.bar(x_labels[i], x_m_values[i],  align='center', alpha=0.6, ecolor='black', capsize=10, color=colors[i])
# ax.set_ylabel('Time delay [ms]')
# ax.set_title('Time delay REST vs gRPC')
# ax.yaxis.grid(True)

import  seaborn as sns
import matplotlib.pyplot as plt
plt.figure('Time delay REST vs gRPC')
plt.title('Time delay REST vs gRPC')

sns.set_style('whitegrid')
sns.barplot(x=x_labels, y=x_m_values, palette='deep')
plt.errorbar(x=x_labels, y=x_m_values, fmt='none', ecolor='black', capsize=10)

plt.ylim(0, 12)



# Show the plot
plt.show()

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1765: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categ

In [106]:
RANGE =[900000, 940000]
df_src_feat_list = []
df_src_feat_list_1 = []
df_src_feat_list_2 = []

for elem in  flow_list[0]['flows']:
    tmp = elem['feat']['ps']
    df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[1]['flows']:
    tmp = elem['feat']['ps']
    df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[2]['flows']:
    tmp = elem['feat']['ps']
    df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 5000])
    
df_src_feat_list_all = []
df_src_feat_list_all_1 = []
df_src_feat_list_all_2 = []

for elem in df_src_feat_list:
    df_src_feat_list_all.append(df_src_feat_list_all)

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1.append(df_src_feat_list_all_1)

for elem in df_src_feat_list_2:
    df_src_feat_list_all_2.append(df_src_feat_list_all_2)
    
name = 'frame.len.sum'

x_m,std_m = extract_mean_std_feat_list(df_src_feat_list,name)
x_m1,std_m1 = extract_mean_std_feat_list(df_src_feat_list_1,name)
x_m2,std_m2 = extract_mean_std_feat_list(df_src_feat_list_2,name)

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': RANGE}},
    value = x_m,
    delta = {'reference': 236534},
    domain = {'x': [0.1, 1], 'y': [0, 0.25]},
    title = {'text': flow_list[0]['type']}))
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': RANGE}},
    value = x_m1,
    delta = {'reference': 236534},
    domain = {'x': [0.1, 1], 'y': [0.35, 0.6]},
    title = {'text': flow_list[1]['type']}))
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': RANGE}},
    value = x_m2,
    delta = {'reference': 236534},
    domain = {'x': [0.1, 1], 'y': [0.7, 0.95]},
    title = {'text': flow_list[2]['type']}))

# update layout
fig.update_layout(height=400,title_text="Overhead introduced by protocol in Bytes per transmission round Parameter Server side")

fig.show()

In [102]:
import seaborn as sns
import matplotlib.pyplot as plt

RANGE =[900000, 920000]
df_src_feat_list = []
df_src_feat_list_1 = []
df_src_feat_list_2 = []

for elem in  flow_list[0]['flows']:
    tmp = elem['feat']['ps']
    df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[1]['flows']:
    tmp = elem['feat']['ps']
    df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[2]['flows']:
    tmp = elem['feat']['ps']
    df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 5000])
    
df_src_feat_list_all = []
df_src_feat_list_all_1 = []
df_src_feat_list_all_2 = []

for elem in df_src_feat_list:
    df_src_feat_list_all.append(df_src_feat_list_all)

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1.append(df_src_feat_list_all_1)

for elem in df_src_feat_list_2:
    df_src_feat_list_all_2.append(df_src_feat_list_all_2)
    
name = 'frame.len.sum'

x_m,std_m = extract_mean_std_feat_list(df_src_feat_list,name)
x_m1,std_m1 = extract_mean_std_feat_list(df_src_feat_list_1,name)
x_m2,std_m2 = extract_mean_std_feat_list(df_src_feat_list_2,name)

x_m_values = [x_m,x_m1,x_m2]
colors = ['blue', 'red', 'green']
std_m_values = [std_m,std_m1,std_m2]
# Create the bar graph plot
# fig, ax = plt.subplots()
# for i in range(len(x_labels)):
#     ax.bar(x_labels[i], x_m_values[i],  align='center', alpha=0.6, ecolor='black', capsize=10, color=colors[i])
# ax.set_ylabel('Bytes')
# ax.set_title('Protocol Overhead per transmission round (Parameter Server)')
# ax.yaxis.grid(True)
# ax.axhline(y=236534, color='red', linestyle='--')


sns.set_style('whitegrid')
fig, axs = plt.subplots(1, 2, figsize=(8, 8))

sns.barplot(x=x_labels, y=x_m_values, palette='deep', ax=axs[0])
axs[0].errorbar(x=x_labels, y=x_m_values,  fmt='none', ecolor='black', capsize=10)

axs[0].axhline(y=236534, color='red', linestyle='--')
axs[0].set_ylabel('Bytes')
axs[0].set_title('Parameter Server')
axs[0].set_ylim(200000, axs[0].get_ylim()[1])

# # Add a red dashed line at the value 236534
# plt.axhline(y=236534, color='red', linestyle='--')
# plt.title('Protocol Overhead per transmission round (Parameter Server)')
# plt.ylim(200000, plt.ylim()[1])
# Show the plot
# plt.show()

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1765: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categ

(200000.0, 981882.9525395661)

In [103]:
RANGE =[275000, 285000]
df_src_feat_list = []
df_src_feat_list_1 = []
df_src_feat_list_2 = []

for elem in  flow_list[0]['flows']:
    tmp = elem['feat']['client']
    df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[1]['flows']:
    tmp = elem['feat']['client']
    df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[2]['flows']:
    tmp = elem['feat']['client']
    df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 5000])
    
df_src_feat_list_all = []
df_src_feat_list_all_1 = []
df_src_feat_list_all_2 = []

for elem in df_src_feat_list:
    df_src_feat_list_all.append(df_src_feat_list_all)

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1.append(df_src_feat_list_all_1)

for elem in df_src_feat_list_2:
    df_src_feat_list_all_2.append(df_src_feat_list_all_2)
    
name = 'frame.len.sum'

x_m,std_m = extract_mean_std_feat_list(df_src_feat_list,name)
x_m1,std_m1 = extract_mean_std_feat_list(df_src_feat_list_1,name)
x_m2,std_m2 = extract_mean_std_feat_list(df_src_feat_list_2,name)

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': RANGE}},
    value = x_m,
    delta = {'reference': 236534},
    domain = {'x': [0.1, 1], 'y': [0, 0.25]},
    title = {'text': flow_list[0]['type']}))
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': RANGE}},
    value = x_m1,
    delta = {'reference': 236534},
    domain = {'x': [0.1, 1], 'y': [0.35, 0.6]},
    title = {'text': flow_list[1]['type']}))
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': RANGE}},
    value = x_m2,
    delta = {'reference': 236534},
    domain = {'x': [0.1, 1], 'y': [0.7, 0.95]},
    title = {'text': flow_list[2]['type']}))

# update layout
fig.update_layout(height=400,title_text="Overhead introduced by protocol in Bytes per transmission round Parameter Client side")

fig.show()

In [104]:
RANGE =[275000, 285000]
df_src_feat_list = []
df_src_feat_list_1 = []
df_src_feat_list_2 = []

for elem in  flow_list[0]['flows']:
    tmp = elem['feat']['client']
    df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[1]['flows']:
    tmp = elem['feat']['client']
    df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[2]['flows']:
    tmp = elem['feat']['client']
    df_src_feat_list_2.append(tmp[tmp['frame.len.sum'] > 5000])
    
df_src_feat_list_all = []
df_src_feat_list_all_1 = []
df_src_feat_list_all_2 = []

for elem in df_src_feat_list:
    df_src_feat_list_all.append(df_src_feat_list_all)

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1.append(df_src_feat_list_all_1)

for elem in df_src_feat_list_2:
    df_src_feat_list_all_2.append(df_src_feat_list_all_2)
    
name = 'frame.len.sum'

x_d,std_m = extract_mean_std_feat_list(df_src_feat_list,name)
x_d1,std_m1 = extract_mean_std_feat_list(df_src_feat_list_1,name)
x_d2,std_m2 = extract_mean_std_feat_list(df_src_feat_list_2,name)

x_m_values_1 = [x_d,x_d1,x_d2]
colors = ['blue', 'red', 'green']
std_m_values = [std_m,std_m1,std_m2]
# Create the bar graph plot
# fig, ax = plt.subplots()
# for i in range(len(x_labels)):
#     ax.bar(x_labels[i], x_m_values[i],  align='center', alpha=0.6, ecolor='black', capsize=10, color=colors[i])
# ax.set_ylabel('Bytes')
# ax.set_title('Protocol Overhead per transmission round (Parameter Server)')
# ax.yaxis.grid(True)
# ax.axhline(y=236534, color='red', linestyle='--')


# sns.set_style('whitegrid')
# sns.barplot(x=x_labels, y=x_m_values, palette='deep')
# plt.errorbar(x=x_labels, y=x_m_values, fmt='none', ecolor='black', capsize=10)
# 
# # Add a red dashed line at the value 236534
# plt.axhline(y=236534, color='red', linestyle='--')
# 
# plt.title('Protocol Overhead per transmission round (Client)')
# plt.ylim(200000, plt.ylim()[1])

sns.barplot(x=x_labels, y=x_m_values_1, palette='deep', ax=axs[1])
axs[1].errorbar(x=x_labels, y=x_m_values_1,  fmt='none', ecolor='black', capsize=10)

axs[1].axhline(y=236534, color='red', linestyle='--')
axs[1].set_ylabel('Bytes')
axs[1].set_title('Client')
axs[1].set_ylim(200000, axs[1].get_ylim()[1])

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.5)

# Show the plot
plt.show()

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1765: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.

/home/antonio/TRUSTroke/FLBenchComm/Framework/FedML/venv/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categ

In [105]:
df_src_feat_list = []
df_src_feat_list_1 = []

for elem in  flow_list[0]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list.append(tmp[tmp['frame.len.sum'] > 5000])

for elem in  flow_list[1]['flows']:
    tmp = pd.concat([elem['feat']['ps'], elem['feat']['client']])
    df_src_feat_list_1.append(tmp[tmp['frame.len.sum'] > 5000])
    
df_src_feat_list_all = []
df_src_feat_list_all_1 = []

for elem in df_src_feat_list:
    df_src_feat_list_all.append(df_src_feat_list_all)

for elem in df_src_feat_list_1:
    df_src_feat_list_all_1.append(df_src_feat_list_all_1)
    
name = 'frame.len.median'

x_m,std_m = extract_mean_std_feat_list(df_src_feat_list,name)
x_m1,std_m1 = extract_mean_std_feat_list(df_src_feat_list_1,name)

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': [1000, 2000]}},
    value = x_m,
    delta = {'reference': 1514},
    domain = {'x': [0.1, 1], 'y': [0, 0.35]},
    title = {'text': flow_list[0]['type']}))
fig.add_trace(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {'shape': "bullet",'axis': {'range': [1000, 2000]}},
    value = x_m1,
    delta = {'reference': 1514},
    domain = {'x': [0.1, 1], 'y': [0.65, 1]},
    title = {'text': flow_list[1]['type']}))

# update layout
fig.update_layout(height=400,title_text="Throughput by protocol REST vs gRPC in Bytes")

fig.show()